In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import re

In [2]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import download
download('punkt_tab')
download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
from nltk.tokenize import sent_tokenize
from nltk.collocations import *

[nltk_data] Downloading package punkt_tab to /Users/lisa/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/lisa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Первый этап — извлечение всех сообщений из Telegram-канала:
-парсинг Telegram-канала по частям, так как объём очень большой (сентябрь–апрель 2022–2023, сентябрь–апрель 2023–2024, сентябрь–апрель 2024–2025, август–январь 2025–2026)
-применяем BeautifulSoup к каждому документу
-извлекаем текст из каждого сообщения 

In [3]:
#прописываем функцию для получения супа из html-файла с сообщениями (в данном случае - из файла с сообщениями из телеграм-канала)
def soup_for_messages(filename, encoding="utf-8"):
    with codecs.open(filename, "r", encoding) as f:
        html_content = f.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup

In [4]:
#прописываем функцию для экстракции текста сообщения 
def extract_messages(soup):
    stop_phrases = [ "добро пожаловать в чат нови сада", "я не робот", "userinfo|bot", "кикнут", "разбаном в настройках чата" ]
    messages = []
    for i in soup.find_all('div', {'class': "body"}):
        text_div = i.find('div', {'class': "text"})
        if text_div is not None:
            res = text_div.text.strip().replace('\n', ' ').lower()
            if any(phrase in res for phrase in stop_phrases): 
                continue
            messages.append(res)
    return messages

In [5]:
all_messages_2022 = []
soup = soup_for_messages("/Users/lisa/Desktop/Linguistics/Проект/сентябрь 22 - апрель 23/messages.html")
all_messages_2022.extend(extract_messages(soup))

for i in range(2, 55):
    file_path = f"/Users/lisa/Desktop/Linguistics/Проект/сентябрь 22 - апрель 23/messages{i}.html"
    
    soup = soup_for_messages(file_path)
    all_messages_2022.extend(extract_messages(soup))

In [6]:
all_messages_2023 = []
soup = soup_for_messages("/Users/lisa/Desktop/Linguistics/Проект/сентябрь 2023 - апрель 2024 /messages.html")
all_messages_2023.extend(extract_messages(soup))

for i in range(2, 77):
    file_path = f"/Users/lisa/Desktop/Linguistics/Проект/сентябрь 2023 - апрель 2024 /messages{i}.html"
    
    soup = soup_for_messages(file_path)
    all_messages_2023.extend(extract_messages(soup))

In [7]:
all_messages_2024 = []
soup = soup_for_messages("/Users/lisa/Desktop/Linguistics/Проект/сентябрь 2024 - апрель 2025 /messages.html")
all_messages_2024.extend(extract_messages(soup))

for i in range(2, 55):
    file_path = f"/Users/lisa/Desktop/Linguistics/Проект/сентябрь 2024 - апрель 2025 /messages{i}.html"
    
    soup = soup_for_messages(file_path)
    all_messages_2024.extend(extract_messages(soup))

In [8]:
all_messages_2025 = []
soup = soup_for_messages("/Users/lisa/Desktop/Linguistics/Проект/август 2025 - январь 2026/messages.html")
all_messages_2025.extend(extract_messages(soup))

for i in range(2, 47):
    file_path = f"/Users/lisa/Desktop/Linguistics/Проект/август 2025 - январь 2026/messages{i}.html"
    
    soup = soup_for_messages(file_path)
    all_messages_2025.extend(extract_messages(soup))

На втором этапе проводим очистку сообщений:
-создаём список стоп-слов (стандартный список + добавленные)
-запускаем функцию, в которой очищаем текст от пунктуации, стоп-слов и слов, отличных от кириллицы
-токенизируем текст

In [17]:
stop_words = stopwords.words('russian')
stop_words = stop_words + ['это', 'всё', 'очень', 'то', 'что', 'с','привет','спасибо','просто','ещё','в','почему','такое', 'подскажите', 'пожалуйста','добрый', 'день','напишите','вечер','доброе', 'утро','скажите','день','всем','вообще','вроде', 'общем','кикнут','кик','разбаном','проголосовали', 'т', 'д','нови', 'сад', 'саде', 'сада', 'какие','например','могу', 'сегодня','знаю', 'нужно','пока','кого', 'кто', 'который','вопрос']

In [18]:
#прописываем функцию для очистки сообщений от пунктуации, стоп-слов и слов, отличных от кириллицы
def clean_messages(messages):
    cleaned=[]
    messages_without_punk=[word for word in messages if word[0].isalnum()]
    messages_cleaned= [word for word in messages_without_punk if word not in stop_words]
    russian_words = [w for w in messages_cleaned if re.fullmatch(r"[А-Яа-яЁё]+", w)]
    cleaned.extend(russian_words)
    return cleaned

In [19]:
#токенизированные и очищенные сообщения по годам
text_2022 = " ".join(all_messages_2022) 
res_2022 = wordpunct_tokenize(text_2022)
messages_cleaned_2022=clean_messages(res_2022)
print(messages_cleaned_2022[:10])

['ребята', 'скиньте', 'номер', 'такси', 'срочно', 'чатика', 'подающих', 'визы', 'сербии', 'кажется']


In [20]:
text_2023 = " ".join(all_messages_2023) 
res_2023 = wordpunct_tokenize(text_2023)
messages_cleaned_2023=clean_messages(res_2023)

In [21]:
text_2024 = " ".join(all_messages_2024) 
res_2024 = wordpunct_tokenize(text_2024)
messages_cleaned_2024=clean_messages(res_2024)

In [22]:
text_2025 = " ".join(all_messages_2025) 
res_2025 = wordpunct_tokenize(text_2025)
messages_cleaned_2025=clean_messages(res_2025)

In [15]:
#пробуем сделать лемматизацию с помощью станзы, но к сожалению, объем документа слишком большой 
#%pip install stanza
#import pandas as pd
#import numpy as np
#import stanza
#stanza.download("ru")
#nlp_stanza = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse, ner")
#lemma_2022=nlp_stanza(text_2022)
#print(lemma_2022[:10])

На третьем этапе проводим анализ корпуса:
-считаем частотные слова для каждого документа (20 самых частотных слов)
-считаем TF-IDF с некоторой поправкой на значимые слова для каждого документа
-выводим топ-20 слов для всего корпуса

In [15]:
#скачиваем каунтер для подсчета количества 
from collections import Counter

In [23]:
#смотрим 20 самых частотных слов в каждом году
word_freq_for_corpus = [Counter(messages_cleaned_2022).most_common(20), Counter(messages_cleaned_2023).most_common(20), Counter(messages_cleaned_2024).most_common(20), Counter(messages_cleaned_2025).most_common(20)]
for result in word_freq_for_corpus:
    print(result, "\n")

[('сербии', 1408), ('рф', 1327), ('евро', 1164), ('нс', 1012), ('внж', 1001), ('купить', 849), ('время', 743), ('работает', 684), ('найти', 669), ('точно', 659), ('месяц', 644), ('россии', 639), ('сколько', 629), ('знает', 583), ('сразу', 573), ('динар', 565), ('думаю', 563), ('стоит', 562), ('именно', 550), ('деньги', 537)] 

[('сербии', 1724), ('внж', 1648), ('рф', 1592), ('нс', 1182), ('евро', 1168), ('купить', 1112), ('время', 1003), ('знает', 954), ('работает', 924), ('сколько', 898), ('точно', 814), ('сразу', 813), ('год', 806), ('найти', 801), ('которые', 769), ('могут', 766), ('деньги', 761), ('стоит', 758), ('кстати', 734), ('года', 730)] 

[('сербии', 1361), ('внж', 1018), ('рф', 967), ('нс', 915), ('евро', 868), ('работает', 808), ('знает', 763), ('время', 758), ('точно', 748), ('года', 714), ('купить', 709), ('которые', 661), ('год', 638), ('могут', 637), ('сколько', 605), ('именно', 582), ('возможно', 581), ('найти', 576), ('чат', 572), ('поэтому', 569)] 

[('сербии', 983)

In [25]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
%pip install scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
%pip install pandas


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#прописываем векторизацию для всего корпуса, объединяя все сообщения в один список
#min_df=2 Минимальная частота документа: слово должно встречаться минимум в 2 документах, чтобы попасть в словарь
#max_df=0.9 Максимальная частота документа: слово должно встречаться не более чем в 90% документов. (подсказал AI с минимальной и максимальной частотой)
all_messages_cleaned = [" ".join(messages_cleaned_2022),
         " ".join(messages_cleaned_2023),
         " ".join(messages_cleaned_2024),
         " ".join(messages_cleaned_2025)]
tfidf = tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, token_pattern=r'\b[а-яА-Я]{3,}\b', min_df=2, max_df=0.9 )
tfidf = tfidf_vectorizer.fit_transform(all_messages_cleaned)
tfidf.shape

(4, 43044)

In [28]:
#смотрим результат в виде датафрейма, где строки — документы, столбцы — слова, значения — TF‑IDF веса.
words = tfidf_vectorizer.get_feature_names_out()
import pandas as pd 
data = tfidf.todense().tolist()
result_tfidf = pd.DataFrame(data, columns = words)
#выведим первые 20 столбцов, чтобы не перегружать вывод
result_tfidf[result_tfidf.columns[:20]]

,аааааа,ааааааа,аааааааа,абакан,абакане,аббревиатурах,аббревиатуру,абзаца,абзаце,абзацев,абонемента,абонементами,абонементе,абонементов,абонементу,абонементы,абонент,абонента,абонентка,абонентки
0,0.005450,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008175,0.000000,0.005450,0.005450,0.000000,0.000000,0.002725,0.006732,0.008175,0.000000,0.000000,0.000000,0.003366
1,0.009530,0.002354,0.000000,0.000000,0.000000,0.002354,0.002354,0.000000,0.004709,0.001906,0.003812,0.002354,0.002354,0.007624,0.002354,0.007624,0.007624,0.001906,0.002354,0.002354
2,0.007159,0.002948,0.002948,0.002948,0.002948,0.000000,0.000000,0.009546,0.002948,0.000000,0.004773,0.000000,0.002948,0.000000,0.000000,0.000000,0.009546,0.007159,0.002948,0.000000
3,0.000000,0.000000,0.006306,0.006306,0.003153,0.003153,0.003153,0.002552,0.000000,0.005105,0.000000,0.003153,0.000000,0.005105,0.000000,0.005105,0.002552,0.005105,0.000000,0.000000


In [29]:
#выведим 20 слов с наибольшим TF-IDF весом для каждого года
tfidf2022 = sorted(zip(tfidf.toarray()[0], words), reverse=True)[:20]
df_tfidf2022 = pd.DataFrame(tfidf2022, columns=["TF-IDF", "Word"])
print(df_tfidf2022)

      TF-IDF          Word
0   0.158049         экспо
1   0.117807           пцр
2   0.089925      элефанте
3   0.084475    знакомимся
4   0.065400         ахмад
5   0.060586          юань
6   0.060586       рыбачук
7   0.060586     машинного
8   0.057225  рассказываем
9   0.057225     лендлорда
10  0.054500     юнистрима
11  0.054500      кулстори
12  0.051775    завтракаем
13  0.049050      переписи
14  0.049050      пакетики
15  0.049050     компонент
16  0.049050       вклейку
17  0.047123       купчино
18  0.047123         англе
19  0.043757     экспобанк


In [30]:
tfidf2023=sorted(zip(tfidf.toarray()[1], words), reverse=True)[:20]
df_tfidf2023 = pd.DataFrame(tfidf2023, columns=["TF-IDF", "Word"])
print(df_tfidf2023)

      TF-IDF         Word
0   0.171546     выключен
1   0.153034        агния
2   0.120082        павлы
3   0.099116          нко
4   0.097209         додо
5   0.091491    модерации
6   0.085773        павлу
7   0.072431    суперверо
8   0.062900         веро
9   0.057182  потребление
10  0.057182   знакомимся
11  0.056505        агнии
12  0.055276  голосование
13  0.053370     зеленило
14  0.051464          лир
15  0.049558        срачи
16  0.049558     площадок
17  0.047652     банофбот
18  0.047087         чото
19  0.045746      вклейку


In [31]:
tfidf2024 = sorted(zip(tfidf.toarray()[2], words), reverse=True)[:20]
df_tfidf2024 = pd.DataFrame(tfidf2024, columns=["TF-IDF", "Word"])
print(df_tfidf2024)

      TF-IDF            Word
0   0.147957        радничка
1   0.131252         таланту
2   0.126479       отключали
3   0.083524            оков
4   0.081138        радничке
5   0.076365            веро
6   0.070744        депревоз
7   0.069206        трагедии
8   0.069206         протест
9   0.066819            июль
10  0.064433           напор
11  0.059660            скам
12  0.059660           дейли
13  0.052501             миф
14  0.052501           грибы
15  0.050111  хахахахахахаха
16  0.050111          бойкот
17  0.047728     потребление
18  0.047728           окове
19  0.047728          никита


In [116]:
tfidf2025 = sorted(zip(tfidf.toarray()[3], words), reverse=True)[:20]
df_tfidf2025 = pd.DataFrame(tfidf2025, columns=["TF-IDF", "Word"])
print(df_tfidf2025)

      TF-IDF           Word
0   0.114862        таланту
1   0.112309            мыш
2   0.081973           юнет
3   0.081679           скам
4   0.078821          боник
5   0.075668          бобик
6   0.071470        йогурта
7   0.069362           кекс
8   0.068917           мыша
9   0.066365       подписку
10  0.066365           кеса
11  0.058707            жпт
12  0.056751            цой
13  0.056751      пейсатель
14  0.056155        таланта
15  0.051050            гпт
16  0.051050          белки
17  0.048497  правительства
18  0.048497            плс
19  0.048497        кальция


In [32]:

#TF-IDF по всем документам
word_scores = np.asarray(tfidf.sum(axis=0)).flatten()
top_words = sorted(zip(word_scores, words), reverse=True)[:20]
df_top_words= pd.DataFrame(top_words, columns=["TF-IDF", "Word"])
print(df_top_words)


      TF-IDF          Word
0   0.276611       таланту
1   0.205852      радничка
2   0.187724      выключен
3   0.184612    знакомимся
4   0.178623     отключали
5   0.165541         экспо
6   0.163132         агния
7   0.160400          скам
8   0.159195          додо
9   0.158778     суперверо
10  0.145275         павлы
11  0.144020  рассказываем
12  0.143681      кулстори
13  0.141884      элефанте
14  0.141818          веро
15  0.136664    завтракаем
16  0.128053           мыш
17  0.127882   потребление
18  0.123237      радничке
19  0.122515           пцр


На четвёртом этапе анализируем коллокации:
-извлекаем биграммы для каждого документа
-анализируем коллокации с помощью student t-test

In [34]:
import nltk
from nltk import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
from IPython.display import display
from pprint import pprint

In [35]:
#прописываем функцию для получения биграмм из списка слов
def ngram_splitter(lst, n):
  for i in range(len(lst) - n + 1):
    yield lst[i:i + n]

In [36]:
#проверяем работу функции на биграммах для 2022 года
bigrams_2022 = list(ngram_splitter(messages_cleaned_2022, 2))
print(bigrams_2022[:10])

[['ребята', 'скиньте'], ['скиньте', 'номер'], ['номер', 'такси'], ['такси', 'срочно'], ['срочно', 'чатика'], ['чатика', 'подающих'], ['подающих', 'визы'], ['визы', 'сербии'], ['сербии', 'кажется'], ['кажется', 'текущих']]


PMI (Pointwise Mutual Information)
Смысл: измеряет, насколько вероятность встретить слова вместе выше, чем ожидалось при их независимом появлении.
Chi-square (χ²)
Смысл: проверяет статистическую значимость связи между словами, сравнивая наблюдаемое распределение с ожидаемым.
Likelihood Ratio (LR)
Смысл: сравнивает две гипотезы:
H₀: слова независимы.
H₁: слова связаны.

In [37]:
#прописываем функцию для получения топ 20 биграм по метрике student_t 
def bigram_analysis(tokens, top_n=20):
    bigrams = list(nltk.ngrams(tokens, 2))
    bigram_counts = Counter(bigrams)
    counts_df = pd.DataFrame(
        [[' '.join(key), key[0], key[1], val] for key, val in bigram_counts.items()],
        columns=['bigram', 'word_1', 'word_2', 'count']
    ).sort_values(by='count', ascending=False, ignore_index=True)
    measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    scored = finder.score_ngrams(measures.student_t)
    student_df = pd.DataFrame(
        [[' '.join([w1, w2]), w1, w2, score] for ((w1, w2), score) in scored],
        columns=['bigram', 'word_1', 'word_2', 'student_t']
    ).sort_values(by='student_t', ascending=False, ignore_index=True)
    
    return {
        'counts': counts_df.head(top_n),
        'student_t': student_df.head(top_n)
    }

In [38]:

# функция для получения топ 20 биграм по метрикам PMI, Chi-square и Likelihood ratio
def bigram_analysis(tokens, top_n=20,min_freq=3):
    measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(min_freq)

    # разные метрики
    scored_pmi = finder.score_ngrams(measures.pmi)
    scored_chi = finder.score_ngrams(measures.chi_sq)
    scored_lr = finder.score_ngrams(measures.likelihood_ratio)

    # подсчёт частот биграмм
    bigram_counts = Counter(zip(tokens[:-1], tokens[1:]))

    def to_df(scored, metric_name):
        return pd.DataFrame(
            [[' '.join([w1, w2]), w1, w2, bigram_counts.get((w1, w2), 0), score]
             for ((w1, w2), score) in scored],
            columns=['bigram', 'word_1', 'word_2', 'count', metric_name]
        ).sort_values(by=metric_name, ascending=False, ignore_index=True).head(top_n)

    return {
        'PMI': to_df(scored_pmi, 'pmi'),
        'Chi-square': to_df(scored_chi, 'chi_sq'),
        'Likelihood ratio': to_df(scored_lr, 'lr')
    }


In [39]:
#результаты скорее просто ради интереса, так как не была проведена лемматизация 
result_2022 = bigram_analysis(messages_cleaned_2022, top_n=20)
print(result_2022['PMI'])
print(result_2022['Likelihood ratio'])
print(result_2022['Chi-square'])

                        bigram         word_1         word_2  count        pmi
0                  анице ребац          анице          ребац      3  17.047848
1           легализуем учебные     легализуем        учебные      3  17.047848
2                     ня кавай             ня          кавай      3  17.047848
3             поломками кончно      поломками         кончно      3  17.047848
4                 птц телекком            птц       телекком      3  17.047848
5   самовольного модерирования   самовольного  модерирования      3  17.047848
6           соседях разузнаете        соседях     разузнаете      3  17.047848
7                  фиш зелениш            фиш        зелениш      3  17.047848
8             веселина маслеше       веселина        маслеше      3  17.047848
9         курьерскими службами    курьерскими       службами      4  16.632811
10            сараевский чевап     сараевский          чевап      4  16.632811
11          плановых внезапных       плановых      в

In [40]:
result_2023 = bigram_analysis(messages_cleaned_2023, top_n=20)
print(result_2023['PMI'])
print(result_2023['Likelihood ratio'])
print(result_2023['Chi-square'])

                      bigram          word_1        word_2  count        pmi
0     адаптировалась украшая  адаптировалась       украшая      3  17.532379
1                 пруд пруди            пруд         пруди      3  17.532379
2                алберт хайн          алберт          хайн      3  17.532379
3          ёлочное украшение         ёлочное     украшение      3  17.532379
4          экстремист фашист      экстремист        фашист      3  17.532379
5       шиномонтаж райончике      шиномонтаж     райончике      3  17.532379
6     чрезмерно загрязненный       чрезмерно  загрязненный      3  17.532379
7                    хум хау             хум           хау      3  17.532379
8            српских сестара         српских       сестара      3  17.532379
9    симфоническим оркестром   симфоническим     оркестром      3  17.532379
10                свака част           свака          част      3  17.532379
11        райончике быстрица       райончике      быстрица      3  17.532379

In [41]:
result_2024 = bigram_analysis(messages_cleaned_2024, top_n=20)
print(result_2024['PMI'])
print(result_2024['Likelihood ratio'])
print(result_2024['Chi-square'])

                        bigram     word_1             word_2  count        pmi
0       азиатско тихоокеанский   азиатско      тихоокеанский      3  17.134844
1               нижним тагилом     нижним            тагилом      3  17.134844
2              царские припасы    царские            припасы      3  17.134844
3                  тошин бунар      тошин              бунар      3  17.134844
4             раннем пубертате     раннем          пубертате      3  17.134844
5         прощение поспешность   прощение        поспешность      3  17.134844
6            приношу извинения    приношу          извинения      3  17.134844
7   омладине круглосуточныйтел   омладине  круглосуточныйтел      3  17.134844
8                обуют ограбят      обуют            ограбят      3  17.134844
9                бухты барахты      бухты            барахты      3  17.134844
10              лигу чемпионов       лигу          чемпионов      3  17.134844
11  качествах профессионализме  качествах   професси

In [49]:
result_2025 = bigram_analysis(messages_cleaned_2025, top_n=20)
print(result_2025['PMI'])
print(result_2025['Likelihood ratio'])
print(result_2025['Chi-square'])

                       bigram         word_1        word_2  count        pmi
0              амерички кутак       амерички         кутак      3  16.902116
1            психиатор натуре      психиатор        натуре      3  16.902116
2      финансовой грамотности     финансовой   грамотности      3  16.902116
3                  туева хуча          туева          хуча      3  16.902116
4        технико пассажирской        технико  пассажирской      3  16.902116
5                судски тумач         судски         тумач      3  16.902116
6                 рядовой дев        рядовой           дев      3  16.902116
7              баним проходим          баним      проходим      3  16.902116
8       проведены проводимого      проведены   проводимого      3  16.902116
9           вырезано цензурой       вырезано      цензурой      3  16.902116
10          бутамирата цитрат     бутамирата        цитрат      3  16.902116
11               острой болью         острой         болью      3  16.902116

На пятом этапе создаём Telegram-бота, чтобы показать, как данный корпус можно использовать дальше:
Telegram-бот выводит сообщения из всего корпуса текстов по заданному слову

In [43]:
import telebot
from telebot import types
import time

In [ ]:
gde_kupit_tvorog_bot = #токен# 
bot = telebot.TeleBot(gde_kupit_tvorog_bot)

In [45]:
all_messages = all_messages_2022 + all_messages_2023 + all_messages_2024 + all_messages_2025

In [47]:
#логика выдачи следующих 5 сообщений с конца была сделана с помощью copilot 
user_sessions = {}

@bot.message_handler(commands=["start"])
def start(message, res=False):
    bot.send_message(message.chat.id,
        "Добрый день!\n"
        "Просто напишите слово кириллицей — и я выведу все сообщения из базы, где оно встречается. Например \"творог\".")

@bot.message_handler(content_types=["text"])
def search_word(message):
    word = message.text.strip().lower() #если уберем lower, то можем вводить словосочетания, но есть нюанс 

    # если пользователь нажал кнопку "Да, пришли"
    if word == "да, пришли" and message.chat.id in user_sessions:
        session = user_sessions[message.chat.id]

        # сначала уменьшаем индекс
        session["index"] -= 5
        start = max(0, session["index"])
        end = start + 5

        next_batch = session["results"][start:end]

        for w in next_batch:
            bot.send_message(message.chat.id, f"📌 {w}")
            time.sleep(1)

        if session["index"] > 0:
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
            item1 = types.KeyboardButton("Да, пришли")
            item2 = types.KeyboardButton("Нет, достаточно")
            markup.row(item1, item2)
            bot.send_message(message.chat.id, "Хотите ещё 5 предыдущих сообщений?", reply_markup=markup)
        else:
            bot.send_message(message.chat.id, "Все сообщения показаны.", reply_markup=types.ReplyKeyboardRemove())
        return

    if word == "нет, достаточно":
        bot.send_message(message.chat.id, "Окей, остановились 👍", reply_markup=types.ReplyKeyboardRemove())
        user_sessions.pop(message.chat.id, None)
        return

    # обычный поиск
    result = [w for w in all_messages if word in w.lower().split()]
    if not result:
        bot.send_message(message.chat.id, "Совпадений не найдено.")
        return

    # сохраняем результаты: начинаем с конца
    user_sessions[message.chat.id] = {"results": result, "index": len(result)}

    # показываем последние 5
    last_batch = result[-5:]
    for w in last_batch:
        bot.send_message(message.chat.id, f"📌 {w}")
        time.sleep(1)

    user_sessions[message.chat.id]["index"] = len(result) - 5
    
    if len(result) > 5:
        bot.send_message(message.chat.id, f"👉 Найдено {len(result)} сообщений. Показаны последние 5.")
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        item1 = types.KeyboardButton("Да, пришли")
        item2 = types.KeyboardButton("Нет, достаточно")
        markup.row(item1, item2)
        bot.send_message(message.chat.id, "Хотите посмотреть еще 5 предыдущих сообщений?", reply_markup=markup)


In [52]:
bot.polling(none_stop=True, interval=0)